In [1]:
# Set up API key and do the necessary imports
import os
from taskgen import *

os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

In [3]:
import os
from typing import List

import pyautogui
from dotenv import load_dotenv
from helium import go_to, kill_browser, start_chrome
from taskgen import Agent


# Example
def add_number_to_list(num1: int, num_list: List[int], *args, **kwargs) -> List[int]:
    """Appends num1 to num_list"""
    num_list.append(num1)
    return num_list


# Define the functions
def open_browser() -> str:
    """Opens the web browser"""
    start_chrome()
    return {"Output": "Web browser opened. Currently on empty page."}


def close_browser() -> str:
    """Closes the web browser"""
    kill_browser()
    return {"Output": "Web browser closed."}


def type_text(text: str) -> str:
    """Types text on the keyboard"""
    pyautogui.typewrite(text)
    return {"Output": f"Typed {text}"}


def enter_key() -> str:
    """Hits the enter key"""
    pyautogui.press("enter")
    return {"Output": "Pressed Enter"}


def navigate_to_url_via_address_bar(url: str) -> str:
    """Navigates to the specified url"""
    go_to(url)
    return {"Output": f"Navigated to {url}"}


# Load the API key
def load_api_key(dotenv_path="../.env"):
    load_dotenv(dotenv_path)
    return os.getenv("OPENAI_API_KEY")


load_api_key()

fn_list_1 = [close_browser, open_browser]
BrowserController = Agent(
    "BrowserController",
    "Opens and closes browsers. Does not navigate to pages",
    model="gpt-4o",
    default_to_llm=False,
).assign_functions(fn_list_1)

fn_list_2 = [type_text, navigate_to_url_via_address_bar, enter_key]
WebUser = Agent(
    "WebUser",
    "Performs user actions within the web browser, navigates to web pages and interacts with web pages.",
    model="gpt-4o",
    default_to_llm=False,
).assign_functions(fn_list_2)



In [4]:
# Boss agent
bossagent = Agent(
    "WebNavigator",
    "Assists user to navigate the web. Open webpage before task, and close webpage after task",
    model="gpt-4o",
    default_to_llm=False,
)

# Assign the agents to the boss agent
agent_list = [BrowserController, WebUser]

# print(Webuser.status())

bossagent.assign_agents(agent_list)
# bossagent.status()


In [5]:
bossagent.reset()
output = bossagent.run("Search 'hello world !' on google. Remember to close the webpage once done.")

Observation: The assigned task is to search "hello world !" on Google and close the webpage once done. No subtasks have been completed yet.
Thoughts: To complete the task, I need to first open a web browser, then navigate to Google and perform the search. After that, I need to close the webpage.
Subtask identified: Open a web browser.
Calling function BrowserController with parameters {'instruction': 'Open a web browser.'}

### Start of Inner Agent: BrowserController ###
Observation: The assigned task is to open a web browser, and no subtasks have been completed yet.
Thoughts: To complete the assigned task, I need to open the web browser as no other actions are required.
Subtask identified: Open the web browser.
Calling function open_browser with parameters {}
> {'output_1': {'Output': 'Web browser opened. Currently on empty page.'}}

Observation: The web browser has been successfully opened and is currently on an empty page.
Thoughts: The assigned task was to open a web browser, which

In [5]:
BrowserController.memory_bank['Function'].memory

In [6]:
WebUser.memory_bank['Function'].memory

In [7]:
bossagent.memory_bank['Function'].memory